# Tutorial: Wulff shapes

In [ ]:
%%capture

import os, re
import siman #program package to manage DFT calculations https://github.com/dimonaks/siman
from siman.calc_manage import smart_structure_read, get_structure_from_matproj
from siman.calc_manage import add, res
# Update configurations
from siman import header
from siman.database import write_database, read_database
from siman.set_functions import read_vasp_sets
from siman.header import db
from siman.header import _update_configuration
_update_configuration('../project_conf.py')
read_database() # read saved database if available
from pydoc import importfile
project_sets = importfile('../project_sets.py')
varset = read_vasp_sets(project_sets.user_vasp_sets, override_global = 1) #read user sets

from siman import thermo

header.PATH2PROJECT = 'icys_2024/tutorial_3_wulff_shapes'
header.PATH2EDITOR = 'notepad.exe'

from matplotlib import rc

import matplotlib.pyplot as plt
%matplotlib inline
# plt.rcParams['figure.figsize'] = [3.0, 2.2]
plt.rcParams['figure.dpi'] = 300



In [ ]:
# Only for this tutorial

import csv
from siman.geo import create_surface2, replic
import numpy as np
import matplotlib.patches as patches
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D

from IPython.core.display import Image, display
from tqdm import tqdm


# DFT calculations 

## Zr3O with SIMAN & Materials Project

In [ ]:
Zr3O = get_structure_from_matproj(mat_proj_id = 'mp-1017')   # SIMAN method to get structure of alpha-zirconium


In [ ]:
# Optimize unit cell 

# add("zr3o", "bulk_eos", 1, it_folder = 'bulk', input_st = Zr3O, calc_method = 'uniform_scale', 
#                n_scale_images=10, scale_region = (-5, 5), run=1, up="up2", cluster = 'razor64')   



In [ ]:
res("zr3o.su", 'bulk_eos', list(range(1,11)) + [100], up="up2", show = 'fit', analys_type = 'fit_a', cluster = 'razor64')


## O2 with Pymatgen & Siman 
Experimenal O2 bond length of 1.2075 $\overset{\circ}{\mathrm{A}}$ is taken from [Computational Chemistry Comparison and Benchmark DataBase](https://cccbdb.nist.gov/exp2x.asp?casno=7782447&charge=0)
 as an inital approximation

In [ ]:
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.vis.structure_vtk import StructureVis


In [ ]:
coords = [[0, 0, 0], [0.12075, 0.0, 0.0]]
lattice = Lattice.from_parameters(a=10.0, b=10.0, c=15.0, alpha=90,
                                  beta=90, gamma=90)
o2_mol = Structure(lattice, ["O", "O"], coords)

## Visualiztion of molecule

In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.visualize import view
from ase.visualize.plot import plot_atoms


In [ ]:
ase_o2 = AseAtomsAdaptor.get_atoms(o2_mol)


In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(5,2))
fig.tight_layout()

# ax1.xaxis.set_major_locator(plt.MaxNLocator(3))
# ax1.yaxis.set_major_locator(plt.MaxNLocator(3))
fontsize = 10

plot_atoms(ase_o2, ax1, radii=0.5, rotation=('0x,0y,0z'))
plot_atoms(ase_o2, ax2, radii=0.5, rotation=('-90x,0y,00z'))

ax1.set_title("XY plane", fontsize=fontsize)
ax1.set_xlabel(r"x-axis, $\mathrm{\AA}$", fontsize=fontsize)
ax1.set_ylabel(r"y-axis, $\mathrm{\AA}$", fontsize=fontsize)
ax1.xaxis.set_tick_params(labelsize=fontsize)
ax1.yaxis.set_tick_params(labelsize=fontsize)

ax2.set_title("XZ plane", fontsize=fontsize)
ax2.set_xlabel(r"y-axis, $\mathrm{\AA}$", fontsize=fontsize)
ax2.set_ylabel(r"z-axis, $\mathrm{\AA}$", fontsize=fontsize)
ax2.yaxis.set_tick_params(labelsize=fontsize)
ax2.xaxis.set_tick_params(labelsize=fontsize)



In [ ]:
from siman.core.structure import Structure

In [ ]:
siman_o2 = Structure().update_from_pymatgen(o2_mol)
siman_o2.magmom = [1.0, 1.0]  # magnetic moments of O atoms

In [ ]:
# add("o2", "o2_box", 1, it_folder = 'bulk', input_st = siman_o2, run=2, up="up2", cluster = 'razor64')   


In [ ]:
res("o2", "o2_box", 1, up="up2", cluster = 'razor64')   


In [ ]:
T_points = 6

T_min = 200
T_max = 1000
T_list = np.linspace(T_min, T_max, T_points) 
P = 0.21 # atmospheres
o_list = []
zr_list = []
e_zr3o = db["zr3o.su", "bulk_eos", 100].e0_at * 4

for T in T_list:
    o_chem =  db["o2", "o2_box", 1].e0_at + (thermo.O2(T=T, c2ev=1, P=P)[0] / 2) + 0.68
    zr_chem = (e_zr3o - o_chem) / 3
    o_list.append(o_chem)
    zr_list.append(zr_chem)
    



In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 1.8) )

fontsize = 7
ax.set_xlabel("Temperature, K", fontsize=fontsize)
ax.set_ylabel("Chemical potential, eV", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

ax.plot(T_list, o_list, '-', c='r', label=r"$\mu_{\mathrm{O}}$")
ax.plot(T_list, zr_list, '-', c='b', label=r"$\mu_{\mathrm{Zr}}$")

# Synthesis temperatures are 240-350 C, see https://doi.org/10.1016/j.jssc.2013.10.023
ylim = ax.get_ylim()
xlim = ax.get_xlim()
ax.set_ylim(ylim[0], ylim[1]*1.05)
ylim = ax.get_ylim()

p = patches.Rectangle((240+273, ylim[0]), (350-240), ylim[1]-ylim[0], linewidth=0, alpha=0.1, facecolor='none', zorder=1)
p.set_color('black')
ax.add_patch(p)
ax.set_xlim([T_min, T_max])

ax.legend(fontsize=fontsize, edgecolor="black")
fig.tight_layout()
fig.show()


## Calculation of slabs with free surfaces

<!-- ![slabs](figures/slabs.png) -->

## Симметричные поверхности

<div>
<center>
<img src="figures/slabs_1.png" width="500" height="500"/>
<center>
</div>

Чтобы найти поверхностные энергии для стехиометрических слэбов, мы используем следующую формулу: 
$$
\gamma_{\mathrm {surf}}=\frac{1}{2 A}\left[E_{\mathrm {slab }}-N \times E_{\mathrm {bulk }}\right],
$$
где $A$ — площадь поверхностного сечения слэба, $E_{\mathrm {slab }}$ — полная энергия слэба; $E_{\mathrm {bulk }}$ — полная энергия элементарной(примитичвной) стехиометрической ячейки; $N$ — количество формульных единиц в слэбе.

## Несимметричные поверхности

<div>
<center>
<img src="figures/slabs_2.png" width="500" height="500"/>
<center>
</div>

    
Для нестехиометрических слэбов: 
$$
\gamma_{\mathrm {surf}}=\frac{1}{2 A}\left[E_{\mathrm {slab }}-E_{\mathrm {bulk }}-\sum_{i} \Delta n_{i} \mu_{i}\right],
$$
где $A$ — площадь поверхностного сечения слэба, $E_{\mathrm {slab }}$ — полная энергия слэба, $E_{\mathrm {bulk }}$ — полная энергия сверхъячейки с составом как в объеме; Индекс $i$ обозначает химические элементы в составе материала. $\Delta n_{i}$ — разница атомов типа $i$ между слэбом и объемной сверъячейкой. $\mu_{i}$ — химический состав элемента $i$. 

## Generate all possible slabs for Zr3O

In [ ]:
from pymatgen.core.surface import get_symmetrically_distinct_miller_indices
from pymatgen.core.surface import SlabGenerator


In [ ]:
zr3o = db['zr3o.su', "bulk_eos", 100].copy().end

In [ ]:
zr3o_pmg = zr3o.convert2pymatgen()

In [ ]:
# Find all symmetrically distinct indices below a certain max-index for a given structure. 
# Analysis is based on the symmetry of the reciprocal lattice of the structure.

mil_list = get_symmetrically_distinct_miller_indices(structure = zr3o_pmg, max_index = 2, return_hkil = False)

In [ ]:
mil_list

In [ ]:
idx_list = []
len_list = []

for mil_idx in tqdm(mil_list):
    slabgen = SlabGenerator(initial_structure = zr3o_pmg, miller_index = mil_idx, min_slab_size = 10, 
                            min_vacuum_size = 10, lll_reduce = True, center_slab = False, primitive = False)

    # Number of terminations for the given Miller index
    slabs = slabgen.get_slabs()
    len_list.append(len(slabs))

    # Current miller index as string 
    mil_cur = [ str(x) for x in mil_idx ]  
    mil_cur = "".join(mil_cur)
    idx_list.append( mil_cur )

    for idx, slab in enumerate(slabs):
        st_cur = zr3o.update_from_pymatgen(slab)
        calc_name = mil_cur + "." + str(idx)

        # If you want to perform all calculations by yourself 
        # add(calc_name, "surface", 1, it_folder = 'surfaces', input_st = st_cur, ngkpt=[2,2,1], run=2, up="up2", cluster = 'razor64')   



In [ ]:
# Visualize data
fig, ax = plt.subplots(figsize=(12, 4))

ax.set_xlabel("Symmetrically distinct Miller index", fontsize=20)
ax.set_ylabel("Number of terminations", fontsize=20)
ax.xaxis.set_tick_params(labelsize=18)
ax.yaxis.set_tick_params(labelsize=18)

plt.bar(idx_list, len_list, color="royalblue", width=0.5)

plt.show()

# Calculated surface energies 

In [ ]:
import pandas as pd

In [ ]:
def parse_chemical_formula(formula):
    # Regular expression to match element symbols followed by an optional number
    pattern = r"([A-Z][a-z]*)(\d*)"
    elements = re.findall(pattern, formula)
    
    # Convert the matches to a dictionary with element counts
    parsed_formula = {}
    for (element, count) in elements:
        parsed_formula[element] = int(count) if count else 1  # Default to 1 if count is missing

    return parsed_formula


In [ ]:
# Chemical potentilas at T=300 K and P=0.21 atm
e_zr3o = db["zr3o.su", "bulk_eos", 100].e0_at * 4

o_chem =  db["o2", "o2_box", 1].e0_at + (thermo.O2(T=300, c2ev=1, P=0.21)[0] / 2) + 0.68
zr_chem = (e_zr3o - o_chem) / 3

chem_list = {"O": o_chem, "Zr": zr_chem}



In [ ]:
# Read already calculated data
data_dft = pd.read_csv('data/zr3o_dft.csv', names=["surface", "energy", "formula", "area"])
data_dft = data_dft[data_dft["surface"] != "o2" ]
data_dft = data_dft[data_dft["surface"] != "zr3o" ]


In [ ]:
# Get data from the pandas dataframe
names_list = []
suf_en_list = []
names_list_dft = []

for index, row in data_dft.iterrows():
    names_list.append(row.values[0])
    name_cur = names_list[-1].split(".")[0]
    if (name_cur not in names_list_dft):
        names_list_dft.append(name_cur)
    
    suf_en_cur = float(row.values[1])          
    suf_area_cur = float(row.values[3]) 
    fml_cur = parse_chemical_formula( row.values[2] ) 

    for el in ["Zr", "O"]:
        suf_en_cur -= fml_cur[el] * chem_list[el]

    suf_en_cur /= 2*suf_area_cur
    suf_en_cur *= header.eV_A_to_J_m

    suf_en_list.append(suf_en_cur)
    

# Sufrace and total energies for DFT 
dir_suf = {}

for idx, name in enumerate(names_list):
    name_cur = name.split(".")[0]
    if name_cur not in dir_suf.keys():
        dir_suf[name_cur] = [suf_en_list[idx]]
    else:
        dir_suf[name_cur].append(suf_en_list[idx])
    

In [ ]:
fig, ax = plt.subplots(figsize=(5, 2.2))
fontsize = 7

ax.set_title("DFT results", fontsize=fontsize+2)
ax.set_ylabel(r"Surface energy, J/m$^2$", fontsize=fontsize+2)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

labels, data_plot = dir_suf.keys(), dir_suf.values()
ax.boxplot(data_plot)

ax.set_xticks(range(1, len(labels) + 1), labels, fontsize=fontsize)
ax.set_xlabel("Surface orientation", fontsize=fontsize+2)

plt.show()


# Calculated Wulff shapes 

In [ ]:
from pymatgen.analysis.wulff import WulffShape
from siman.analysis import wulff

In [ ]:
def parse_mil_dir_to_list(mil_dir):
    # Parse list of miller indices (list of str) to list (2D array)  
    mil_list = []
    
    for mil in mil_dir.keys():
        mils_cur = []
        mil_cur = ""
        for char in mil:
            if (char == "-"):
                mil_cur += char
            else:
                mil_cur += char
                mils_cur.append(int(mil_cur))
                mil_cur = ""
        mil_list.append(mils_cur)

    return mil_list

In [ ]:
st = db["zr3o.su", "bulk_eos", 100].copy().end
zr3o_pmg = st.convert2pymatgen()

In [ ]:
# MINIMAL SURFACE ENERGY FOR EACH MILLER INDEX
suf_en_min = {}

for key in dir_suf.keys():
    suf_en_min[key] = min(dir_suf[key])
    
# VISUALIZE WULLF SHAPES
ax = WulffShape(zr3o_pmg.lattice, parse_mil_dir_to_list(suf_en_min),
                         list(suf_en_min.values()) ).get_plot(show_area=False, 
                         aspect_ratio=(8, 16), direction=(1,1,0.2))

# Surface as in rticle 
ax = WulffShape(zr3o_pmg.lattice, [[0,0,1], [1,0,2], [1,1,0], [1,1,1]],
                         [suf_en_min["001"], suf_en_min["102"], 
                        suf_en_min["110"], suf_en_min["111"]] ).get_plot(show_area=False, 
                         aspect_ratio=(8, 16), direction=(1,1,0.2))

# Reference from the article (https://doi.org/10.1016/j.commatsci.2021.110592) 
ax = WulffShape(zr3o_pmg.lattice, [[0,0,1], [1,0,2], [1,1,0], [1,1,1], ],
                         [1.78, 1.87, 1.78, 1.78]  ).get_plot(
                         show_area=False, aspect_ratio=(8, 16), direction=(1,1,0.2))



## All together
Three Wulff shapes: calculated for all Miller indices, calclated only for specific miller indices as that from another work by Boev et. al, 2021 (https://doi.org/10.1016/j.commatsci.2021.110592).

<div>
<center>
<img src="figures/wulff_shapes/wulff_all.png" width="900" height="200"/>
<center>
</div>



# Wulff shapes over temperature

Here, we had stochiometric slab's with two components. As both Zr and O chemical potentials are dependent due to the synthesis environment, Wulff shapes are temperature independent.

However, if a structure has nonstochiometric slabs or more than two components, it will have temperature-dependent Wulff shapes. For instance, MgO from the work by Geysermans et. al, 2009 (https://pubs.rsc.org/en/content/articlelanding/2009/cp/b812376d).


<div>
<center>
<img src="figures/wulff_shapes/wulff_shapes_over_temperature.jpg" width="400" height="400"/>
<center>
</div>
    
<center>
<b>Figure 1.</b>Wulff equilibrium shapes of MgO crystallites as a function ofthe temperature and pressure. Light grey: clean MgO(100) facets, medium grey: hydroxylated MgO(111) facets, dark grey: hydroxylated MgO(100) facets. 
<center>

<br>

<div>
<center>
<img src="figures/wulff_shapes/tem_mgo.jpg" width="222" height="222"/>
<center>
</div>

<center>
<b>Figure 2.</b> TEM images of MgO smoke crystallites: (a) as prepared; mostcubic crystals are seen as projections on (100) planes (orientations oftheir facets); (b) After two days in water (pH = 6.8) at room temperature. Projections are on (100) planes; (c) After seven days in water.
<center>




## Wulff shapes with SIMAN package

In [ ]:

st_wulff = db["zr3o.su", "bulk_eos", 100].copy().end
if 1:
    wulff_shape = wulff(st_wulff, miller_list = [[0,0,1], [1,0,2], [1,1,0], [1,1,1]], 
                                  e_surf_list = [1.780, 1.870, 1.780, 1.780], show=1)

file_name = "figures/your_wulff_shape"
# fig.savefig(file_name+".png", bbox_inches='tight', dpi=600)


In [ ]:
# pip install wulffpack

In [ ]:
from wulffpack import (SingleCrystal,
                       Decahedron,
                       Icosahedron,
                        )

from mpl_toolkits.mplot3d import Axes3D

from ase.build import bulk
from ase.io import write


In [ ]:
surface_energies = {(0, 0, 1): 1.1,
                    (1, 0, 2): 1.0,
                    (1, 1, 0): 1.15,
                    (1, 1, 1): 1.14}

particle = SingleCrystal(surface_energies=surface_energies)



In [ ]:
fig = plt.figure(figsize=(2, 2))
ax = fig.add_subplot(111, projection='3d')

particle.make_plot(ax,)
ax.legend(loc=(1, 0.8), fontsize=5)


## Wulff shapes with [Materials Project](https://next-gen.materialsproject.org/api) API

In [ ]:
from pymatgen.ext.matproj import MPRester
mpr = MPRester("LTSM6dStBrl69FjopxP7KdZBP35B1yh7")  # use this line if you set up the configuration file


In [ ]:
# Get the Wulff shape of a material (currently available for selected elements only)
ws = mpr.get_wulff_shape("mp-135")  # mp-135 is Li



In [ ]:
fig = plt.figure(figsize=(1, 2))  # Width x Height in inches
ax = ws.get_plot()

ax.figure.set_size_inches(5, 15, forward=True)

plt.figure(figsize=(2,1))
plt.show()



In [ ]:
write_database()